In [1]:
# this stage is responsible for processing the data
# pre-requsite is the data should be downloaded... locally.
!pwd

/teamspace/studios/this_studio


In [2]:
import os
os.chdir("100-class-image-classifaction")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PreprocessConfig:
    path: str
    rescale: float
    shear_range: float
    zoom_range: float
    width_shift_range: float
    height_shift_range: float
    horizontal_flip: bool
    validation_split: float
    fill_mode: str
    batch_size: int

In [7]:
from src.LCIC.constants import *
from src.LCIC.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    def __init__(self, 
            config_path: Path = CONFIG_FILE_PATH, 
            params_path: Path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

    def get_preprocess_config()->PreprocessConfig:
        
        preprocessed_cfg = self.config.data_preporcess

        logger.info(f"Preprocessing configurations: ---> {preprocessed_cfg}")
        _cfg=PreprocessConfig(
            path= self.config.data_ingestion.local_data_file  # locally downloade data path
            rescale= preprocessed_cfg.rescale,
            shear_range= preprocessed_cfg.shear_range,
            zoom_range= preprocessed_cfg.zoom_range,
            width_shift_range= preprocessed_cfg.width_shift_range,
            height_shift_range= preprocessed_cfg.height_shift_range,
            horizontal_flip= preprocessed_cfg.horizontal_flip,
            validation_split= preprocessed_cfg.validation_split,
            fill_mode= preprocessed_cfg.fill_mode,
            batch_size= preprocessed_cfg.batch_size
        )
        return _cfg

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
class DataPreprocessing():
    def __init__(slef, config: PreprocessConfig, params_path: Path = PARAMS_FILE_PATH):
        self.config = config
        self.params = read_yaml(params_path)
        
    def __generator(self)->:
        _train_datagen = ImageDataGenerator(
            rescale= self.config.rescale,
            shear_range= self.config.shear_range,
            zoom_range= self.config.zoom_range,
            width_shift_range= self.config.width_shift_range,
            height_shift_range= self.config.height_shift_range,
            horizontal_flip= self.config.horizontal_flip,
            validation_split= self.config.validation_split,
            fill_mode= self.config.fill_mode,
            batch_size= self.config.batch_size
        )
        
        _test_datagen = ImageDataGenerator(
            rescale=1./255)

        return _train_datagen, _test_datagen

    def get_train_and_valid_set():
        train_datagen, val_datagen = self.__generator()
        
        training_set = train_datagen.flow_from_directory(
        directory= self.config.path,
        target_size=[],
        color_mode='rgb',
        classes=None,
        class_mode='categorical',
        batch_size=32,
        shuffle=True,
        seed=None,
        save_to_dir=None,
        save_prefix='',
        save_format='png',
        follow_links=False,
        subset=None,
        interpolation='nearest',
        keep_aspect_ratio=False
        subset="training"
        )

        validation_set = val_datagen.flow_from_directory(
            
        )

        return training_set, validation_set 